In [1]:
import matplotlib.pyplot as plt
import numpy as np
import os
import sys
from matplotlib import colors as mcolors
import torch
import torch.nn as nn

import random
import time
from sklearn.manifold import TSNE
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from torchvision import datasets, models, transforms
from sklearn.decomposition import PCA, FastICA
import itertools
import yaml
from argparse import Namespace
sys.path.append('../src')
import tforms
import feature_extraction.feature_extraction_utils as futils
from feature_extraction.Network_Latents_Wrapper import NetworkLatents

import datasets_utils as dsetutils
import utils

sys.path.append('../src/novelty_dfm_CL')

import novelty_dfm_CL.datasets_holdout_validation as dseth
import novelty_dfm_CL.novelty_detector as novel
import novelty_dfm_CL.novelty_eval as novelval 
import novelty_dfm_CL.classifier as clf
import novelty_dfm_CL.novelty_utils as novelu
import novelty_dfm_CL.incDFM_w_validation as novelinc
import novelty_dfm_CL.novelty_dataset_wrappers as dwrap


np.random.seed(42)

# os.environ['CUDA_LAUNCH_BLOCKING'] = "1"

# parameters

device = 0
ood_method='odin'
dset_id_name = 'cifar100'
dset_od_name = 'svhn'
extractor_name = 'resnet50_contrastive'
finetune_backbone=False


num_outputs_map = {'cifar10':10, 'svhn':10, 'cifar100':100}


num_classes_id = num_outputs_map[dset_id_name]
num_classes_od = num_outputs_map[dset_od_name]


# ------------------------------------------------------



dir_save='/lab/arios/ProjIntel/incDFM/sandbox/results_ood_sup_2d/OOD_%s_ID_%s_OD_%s/'%(ood_method, dset_id_name, dset_od_name)
utils.makedirectory(dir_save)

def get_features(dataset, network_inner, device=0):

    loader = torch.utils.data.DataLoader(dataset, batch_size=100,
                                            shuffle=True, num_workers=4)
    start = time.time()
    print('feat extraction begin')
    current_features = futils.extract_features(network_inner, loader, \
            target_ind=1, homog_ind=-2, device=device)

    print('feat extraction done', time.time()-start)
    feat_name = 'base.8'

    X = current_features[0][feat_name]
    Y = current_features[-2]

    return X, Y, current_features



## Include validation set 
network = clf.Resnet(num_classes_id, resnet_arch=extractor_name, FC_layers=[],  
            resnet_base=-1, multihead_type='single', base_freeze=True, pretrained_weights=None)
network = network.to(device)
# run extractor 
network_inner = NetworkLatents(network, ['base.8'], pool_factors={'base.8':-1})


load contrastive backbone
Will fetch activations from:
base.8, average pooled by -1


# Prepare ID and OD data

In [2]:

experiment_name='MT_Supplementary'
data_dir = '/lab/arios/ProjIntel/incDFM/data/'
experiment_dir='/lab/arios/ProjIntel/incDFM/src/novelty_dfm_CL/Experiments_DFM_CL/'

holdout_percent=0.001
val_percent=0.1


data_ID = dseth.call_dataset_holdout_w_validation(dset_id_name, data_dir, experiment_dir, 
                                        experiment_filepath=None, experiment_name=experiment_name, 
                                        holdout_percent=holdout_percent,  val_holdout=val_percent, scenario='nc', 
                                        num_per_task=num_classes_id, num_classes_first=num_classes_id, 
                                        num_tasks=1, 
                                        shuffle=False, preload=False, keep_all_data=False, \
                                            equalize_labels=False, clip_labels=False, clip_max=15000)


train_dataset_ID, train_holdout_dataset_ID, val_dataset_ID, test_dataset_ID, list_tasks_ID, list_tasks_targets_ID, dset_prep_ID = data_ID
print(train_holdout_dataset_ID[0].__len__(), train_dataset_ID[0].__len__(), val_dataset_ID[0].__len__(), test_dataset_ID[0].__len__())
train_dataset_ID, train_holdout_dataset_ID, val_dataset_ID, test_dataset_ID = train_dataset_ID[0], train_holdout_dataset_ID[0], val_dataset_ID[0], test_dataset_ID[0]




data_OD = dseth.call_dataset_holdout_w_validation(dset_od_name, data_dir, experiment_dir, 
                                        experiment_filepath=None, experiment_name=experiment_name, 
                                        holdout_percent=holdout_percent,  val_holdout=val_percent, scenario='nc', 
                                        num_per_task=num_classes_od, num_classes_first=num_classes_od, 
                                        num_tasks=1, 
                                        shuffle=False, preload=False, keep_all_data=False, \
                                            equalize_labels=False, clip_labels=False, clip_max=15000)

train_dataset_OD, train_holdout_dataset_OD, val_dataset_OD, test_dataset_OD, list_tasks_OD, list_tasks_targets_OD, dset_prep_OD = data_OD

print(train_holdout_dataset_OD[0].__len__(), train_dataset_OD[0].__len__(), val_dataset_OD[0].__len__(), test_dataset_OD[0].__len__())

train_dataset_OD, train_holdout_dataset_OD, val_dataset_OD, test_dataset_OD = train_dataset_OD[0], train_holdout_dataset_OD[0], val_dataset_OD[0], test_dataset_OD[0]



seq_tasks [[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]] {0: 0, 1: 1, 2: 2, 3: 3, 4: 4, 5: 5, 6: 6, 7: 7, 8: 8, 9: 9, 10: 10, 11: 11, 12: 12, 13: 13, 14: 14, 15: 15, 16: 16, 17: 17, 18: 18, 19: 19, 20: 20, 21: 21, 22: 22, 23: 23, 24: 24, 25: 25, 26: 26, 27: 27, 28: 28, 29: 29, 30: 30, 31: 31, 32: 32, 33: 33, 34: 34, 35: 35, 36: 36, 37: 37, 38: 38, 39: 39, 40: 40, 41: 41, 42: 42, 43: 43, 44: 44, 45: 45, 46: 46, 47: 47, 48: 48, 49: 49, 50: 50, 51: 51, 52: 52, 53: 53, 54: 54, 55: 55, 56: 56, 57: 57, 58: 58, 59: 59, 60: 60, 61: 61, 62: 62, 63: 63, 64: 64, 65: 65, 66: 66, 67: 67, 68: 68, 69: 69, 70: 70, 71: 71, 72: 72, 73: 73, 74: 74, 75: 75, 76: 76, 77: 77, 78: 78, 79: 79, 80: 80, 81: 81, 82: 82, 83: 83, 84: 84, 85: 85, 86: 86, 87: 87, 88: 88, 89: 89, 90: 90, 91: 91, 92: 92, 93: 93, 94: 94, 95: 95, 96: 96, 97: 97, 98: 98, 99: 99}
seq_tasks_targets [[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]]
seq_tasks:  [[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]] 1
*****Prep Data*****
Create New Experiment
num_samples_task 5000
/lab/arios/Pro

# OOD Method

In [4]:


name_experiment='OOD_only2_ID_%s_OD_%s'%(dset_id_name,dset_od_name)


ood_config_paths = {'dfm':'/lab/arios/ProjIntel/incDFM/src/configs_new/ood_method/dfm_ood_config.yaml',
                    'incdfm':'/lab/arios/ProjIntel/incDFM/src/configs_new/ood_method/incdfm_ood_config.yaml',
                    'mahal':'/lab/arios/ProjIntel/incDFM/src/configs_new/ood_method/mahal_ood_config.yaml',
                    'odin':'/lab/arios/ProjIntel/incDFM/src/configs_new/ood_method/odin_ood_config.yaml',
                    'softmax':'/lab/arios/ProjIntel/incDFM/src/configs_new/ood_method/softmax_ood_config.yaml',
}

ood_config = ood_config_paths[ood_method]



with open(ood_config) as fid:
    params = Namespace(**yaml.load(fid, Loader=yaml.SafeLoader))
        

# Set up detector params 
params.detector_params['target_ind']=1
params.detector_params['device']=device
params.detector_params['num_classes']=num_classes_id
if ood_method=='odin':
    params.detector_params['base_network']=network #simple network (not wrapper) - Is this problematic? TODO
    params.detector_params['num_classes_fine_tasks']=10
    params.detector_params['criterion']= nn.CrossEntropyLoss()
    params.detector_params['num_epochs']=20
    params.detector_params['train_technique']=1
    params.detector_params['lr']=0.001
    params.detector_params['patience_lr'] = 0.25
    params.detector_params['schedule_decay'] = 0.5
    params.detector_params['step_size_epoch_lr']= 7
    params.detector_params['gamma_step_lr']= 0.1
elif ood_method=='softmax':
    params.detector_params['base_network']=network #simple network (not wrapper) - Is this problematic? TODO
    params.detector_params['num_classes_fine_tasks']=10
    
    

print('Get scores for novelty detector')
noveltyResults = novelval.save_novelty_results(1, name_experiment, dir_save)
# train_loaders_new_only evaluates only current new data (unseen by classifier/main model). 
params_score = {'layer':'base.8', 'feature_extractor':network_inner, 'base_apply_score':True, 'target_ind':1}
# print('new', next(iter(train_loaders_new_only[t])))

novelty_detector = novel.NoveltyDetector().create_detector(type=ood_method, params=params.detector_params)


Get scores for novelty detector


# Fit

In [5]:
import torch.optim as optim

#Test loader
val_loader_ID = torch.utils.data.DataLoader(val_dataset_ID,\
            batch_size=100, shuffle=True, num_workers=4)

print('Fitting %s on %s'%(ood_method, dset_id_name))

if ood_method=='dfm' or ood_method=='mahal' or ood_method=='incdfm':
    print('get features ID - %s'%dset_id_name)
    # Training - Only ID -------------
    Feats_ID_train = get_features(train_dataset_ID, network_inner, device=device)
    novelty_detector.fit_total(Feats_ID_train[0].T, Feats_ID_train[1])
else:
    from novelty_dfm_CL.train_main import train_main_epoch
    from novelty_dfm_CL.test_main import test_main
    # train Loader
    train_loader_ID = torch.utils.data.DataLoader(dwrap.NovelTask(0, num_classes_id, train_dataset_ID, use_coarse=False), \
                batch_size=100, shuffle=True, num_workers=4)
    
    # train classifier jointly 
    if finetune_backbone==True:
        network_inner.model.base.train(True)
        network_inner.base_freeze = False
        for param in network_inner.model.base.parameters():
            param.requires_grad = True
    else:
        network_inner.model.base.train(False)
        network_inner.base_freeze = True
        for param in network_inner.model.base.parameters():
            param.requires_grad = False

    if ood_method=='odin':
        clf_parameters = []
        for name, parameter in network_inner.model.named_parameters():
            if name == 'h.h.weight' or name == 'h.h.bias':
                pass
            else:
                clf_parameters.append(parameter)
    else:
        clf_parameters = network_inner.model.parameters()

        
    optimizer_main = optim.Adam(filter(lambda p: p.requires_grad, clf_parameters), lr=0.001)
    scheduler_main = optim.lr_scheduler.ReduceLROnPlateau(optimizer_main, 'min', patience=0.25, factor=0.5, min_lr=0.00001)
    
    # ----- relative batchsizes between old and new 
    for epoch in range(10):
        print('##########epoch %d###########'%epoch)
        network_inner.model.train()
        if finetune_backbone==True:
            network_inner.model.base.train(True)
            network_inner.base_freeze = False
            for param in network_inner.model.base.parameters():
                param.requires_grad = True
        else:
            network_inner.model.base.train(False)
            network_inner.base_freeze = True
            for param in network_inner.model.base.parameters():
                param.requires_grad = False
                
        train_main_epoch(epoch, train_loader_ID, None, network_inner, optimizer_main,  device=device, OOD_class=novelty_detector,\
                    feature_name='base.8', weight_old=0.5, cut_epoch_short=False,\
                    quantizer=None, target_ind=1, cuda=True, display_freq=10)

        val_loss = test_main(epoch, 0, [val_loader_ID], network_inner, novelty_detector, dir_save, \
            feature_name='base.8', target_ind=1, \
                quantizer=None, cuda=True, device=device)

        # Update schedulers 
        scheduler_main.step(val_loss)


        if novelty_detector.name == 'odin':
            novelty_detector.h_scheduler.step(val_loss)


Fitting odin on cifar100
##########epoch 0###########
Number batches:  45
Train Epoch: 0 [0/45 (0.000)]	Loss: 2.2954	Acc_Train_new:0.090
Train Epoch: 0 [10/45 (22.222)]	Loss: 2.1805	Acc_Train_new:0.150
Train Epoch: 0 [20/45 (44.444)]	Loss: 2.1116	Acc_Train_new:0.220


# Test 

In [ ]:
current_dset = dwrap.CurrentTask(test_dataset_OD, [test_dataset_ID])
current_loader = torch.utils.data.DataLoader(current_dset, batch_size=100,
                                                shuffle=False, num_workers=4)

print('current_dset.__len__()', current_dset.__len__())

current_dset.__len__() 25950


In [ ]:
args=Namespace()
params_score['device']=device
print('Computing Scores for ID %s using %s'%(dset_od_name, ood_method))
if ood_method=='dfm' or ood_method=='odin' or ood_method=='softmax' or ood_method=='mahal':
    
    # 2) Threshold Novel/Old - Binary prediction 
    if ood_method=='odin':
        results_novelty = novelval.evaluate_odin_CL(1, novelty_detector, noveltyResults, params_score, current_loader)
        # invert scores for ODIN so that high scores are novelty (1)
    else:
        results_novelty = novelval.evaluate_simple_CL(1, novelty_detector, noveltyResults, params_score, current_loader, ood_method)

else:
    args.params_score = Namespace(**params_score)
    args.num_samples = current_dset.__len__()
    args.w = 0.5
    args.alg_dfm='simple'
    args.max_iter_pseudolabel=10
    args.threshold_percentile=85
    current_old_new_ratio = 1.0
    args.validation_iid_loader = val_loader_ID
    args.percentile_val_threshold=95
    args.batchsize_test=100
    args.num_workers=4
    args.novelty_detector_name=ood_method
    args.detector_params = params.detector_params
    args.dset_prep = dset_prep_ID
    args.device=device
    args.use_image_as_input = False
    args.add_tf=None
    args.dfm_layers_input = 'base.8'
    # Threshold_n_Select_Iters(params)
    if args.alg_dfm == 'simple':
        SelectTh = novelinc.Threshold_n_Select_Iters(args)
    # elif args.alg_dfm == 'tug':
    #     SelectTh = novelinc.Threshold_Tug_incDFM(args)
        
    inds_pred_novel,_ = SelectTh.select_novel(1, current_dset, novelty_detector, noveltyResults)
    results_novelty = SelectTh.evaluate_CL(1, current_dset, novelty_detector, noveltyResults)
    

Computing Scores for ID svhn using odin
********Score func: h *************
Noise magnitude 0.00250


 Scores | Processing image batch 260/260: 100%|██████████| 260/260 [00:44<00:00,  5.87it/s]


scores [-0.12705427 -0.09504923 -0.14548695 ... -0.22997114 -0.33823946
 -0.33759266]
AUROC: 0.8332665266457681 aupr 0.836433093078297 aupr_norm 0.836433093078297
Noise magnitude 0.00500


 Scores | Processing image batch 260/260: 100%|██████████| 260/260 [00:44<00:00,  5.85it/s]


scores [-0.13559569 -0.10476782 -0.15349543 ... -0.23812331 -0.34583488
 -0.34601322]
AUROC: 0.8359275172413793 aupr 0.8389812704875125 aupr_norm 0.8389812704875125
Noise magnitude 0.01000


...

scores [-0.18616922 -0.16224048 -0.20100628 ... -0.28604779 -0.39020824
 -0.39495802]
AUROC: 0.8504247460815046 aupr 0.8539970943606201 aupr_norm 0.8539970943606201
Noise magnitude 0.04000


 Scores | Processing image batch 260/260: 100%|██████████| 260/260 [00:44<00:00,  5.83it/s]


scores [-0.25059289 -0.23491056 -0.26187825 ... -0.34625694 -0.44528419
 -0.45485133]
AUROC: 0.8655822978056426 aupr 0.8720779775366894 aupr_norm 0.8720779775366894
Noise magnitude 0.08000


 Scores | Processing image batch 260/260: 100%|██████████| 260/260 [00:44<00:00,  5.84it/s]


scores [-0.36288959 -0.35838595 -0.3694981  ... -0.44886035 -0.53721809
 -0.55136108]
AUROC: 0.8842161285266459 aupr 0.896161028033454 aupr_norm 0.896161028033454
New task 1, AUROC = 0.884, AUPR = 0.896, AUPR_NORM = 0.896
new_scores [-0.36288959 -0.35838595 -0.3694981  ... -0.3146278  -0.36126465
 -0.35164791]
old_scores [-0.3900716  -0.52445227 -0.61952615 ... -0.44886035 -0.53721809
 -0.55136108]
best scores [-0.36288959 -0.35838595 -0.3694981  ... -0.44886035 -0.53721809
 -0.55136108]
ODIN Best Values [Score_func h -Noise 0.080]--> Best overall: Auroc 0.884, Aupr 0.896, Aupr_norm 0.896
